## Finding all users that have spam flags and deleting them from within salesforce

In [2]:
## Parameters
#hours_range = 24*365*1 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",   
} 


In [1]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Find all users that have block tags from salesforce


In [5]:
from DB2SalesforceAPI import DB2SalesforceAPI
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [10]:
# get Salesforce ID for organizations
sf_contact_df = db_s.query_data('SELECT Id, firstname, lastname, nanoHUB_user_ID__c,nanoHUB_account_BLOCKED__c FROM Contact')

# display
sf_contact_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000OpRJ0AAN
{"id":"7505w00000OpRJ0AAN","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-18T22:35:08.000+0000","systemModstamp":"2020-08-18T22:35:08.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000OpRJ0AAN","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-18T22:35:08.000+0000","systemModstamp":"2020-08-18T22:35:09.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000OpRJ0AAN","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-18T22:35:08

,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c
0,hubrepo,0035w000031Vsp1AAC,hubrepo,False,998.0
1,nanoHUB,0035w000031Vsp2AAC,support,False,1683.0
2,Grid,0035w000031Vsp3AAC,Statistics,False,1684.0


In [11]:
display(sf_contact_df.tail(3))

,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c
225059,Lucas,0035w00003BRmrsAAD,GC,False,297165.0
225060,Kilolilnolth,0035w00003BRmrtAAD,Kilolilnolth,False,297166.0
225061,NelsonFauri,0035w00003BRmruAAD,NelsonFauri,False,297167.0


In [13]:
## finding the spam IDs and non-spam IDs
spam_rows = sf_contact_df[sf_contact_df['nanoHUB_account_BLOCKED__c'].astype(str).str.contains("True")]
good_rows = sf_contact_df[~sf_contact_df['nanoHUB_account_BLOCKED__c'].astype(str).str.contains('True')]
display(spam_rows.head(2))
display(good_rows.tail(2))

,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c
436,ÃÂÃ¢â¬ÂºÃâÃ¢â¬ËÃâÃâ ÃÂÃÂ°,0035w000031VtFeAAK,ÃÂÃ Â¡ÃÂÃÂ°ÃÂÃÂ¿ÃâÃâÃâÃÂÃâ...,True,43997.0
1219,Jeffrey,0035w000031VtSMAA0,Corgan,True,65047.0


,FirstName,Id,LastName,nanoHUB_account_BLOCKED__c,nanoHUB_user_ID__c
225060,Kilolilnolth,0035w00003BRmrtAAD,Kilolilnolth,False,297166.0
225061,NelsonFauri,0035w00003BRmruAAD,NelsonFauri,False,297167.0


## delete the spam contacts

In [118]:
%load_ext autoreload
%autoreload 2
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'Contact'
#db_s.external_id = 'Id'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Obtained Salesforce access token ...... True


In [119]:
# extract only the ids from spam_rows
spam_ids = spam_rows['Id'].to_frame().reset_index().iloc[:,1].to_frame()

display(spam_ids.head(2))
print(type(spam_ids))

,Id
0,0035w000031VtFeAAK
1,0035w000031VtSMAA0


<class 'pandas.core.frame.DataFrame'>


In [120]:
#spam_ids.to_csv('test.csv',index=False,line_terminator='\n',header=False)
spam_ids.iloc[:1]

,Id
0,0035w000031VtFeAAK


In [121]:
# send data to Salesforce
db_s.delete_data(spam_ids)

[Success] Bulk job creation successful. Job ID = 7505w00000OpRioAAF
hello
[Success] CSV upload successful. Job ID = 7505w00000OpRioAAF
[Success] Closing job successful. Job ID = 7505w00000OpRioAAF


In [125]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000OpRioAAF',
 'operation': 'delete',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-18T23:00:34.000+0000',
 'systemModstamp': '2020-08-18T23:03:23.000+0000',
 'state': 'JobComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 8050,
 'numberRecordsFailed': 1,
 'retries': 0,
 'totalProcessingTime': 116062,
 'apiActiveProcessingTime': 107338,
 'apexProcessingTime': 146097}

In [126]:
db_s.check_bulk_failed_results()

'"sf__Id","sf__Error",Id\n"","ENTITY_IS_DELETED:entity is deleted:--","0035w000031VtFeAAK"\n'